# Online Banking Assistant

This application is the Bank of Transformers online banking application.

Using this application users can:

- open an account
- add funds to an account
- withdraw funds from an account
- check the balance of accounts


In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [3]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [64]:
WELCOME = """\
**Welcome to the Bank of Transformers Online Banking App**

You can do the following:
- open an account
- deposit funds
- withdraw funds
- check the balance of your account

What would you like to do today?
"""

SYS_PROMPT = """\
You are a helpful banking assistant for an online banking app. Your job:
1) Understand the user's intent (open account, deposit, withdraw funds, check balance).
2) If you need to take an action, use the provided tools (functions).
3) Be concise, ask only for the minimum needed info next.
4) Always reflect current balances when relevant.
5) Never invent accounts or balances—rely on tool results only.
6) After carrying out every action for the user, list all available options, as user might want to deposit/withdraw more funds 
"""

In [67]:
open_account_function = {
    "name": "open_account",
    "description": "Open an account for the user. Ask the user to enter their name and email address.",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The name of the user",
            },
            "email": {
                "type": "string",
                "description": "The email address of the user",
            },
        },
        "required": ["name", "email"],
        "additionalProperties": False
    }
}

deposit_funds_function = {
    "name": "deposit_funds",
    "description": "Deposit funds into the user account. Ask the user to enter the amount to deposit.",
    "parameters": {
        "type": "object",
        "properties": {
            "amount": {
                "type": "number",
                "description": "The amount to deposit",
            },
            "balance": {
                "type": "number",
                "description": "The account balance before the deposit",
            }
        },
        "required": ["amount", "balance"],
        "additionalProperties": False
    }
}

withdraw_funds_function = {
    "name": "withdraw_funds",
    "description": "Withdraw funds from the user account. Ask the user to enter the amount to withdraw.",
    "parameters": {
        "type": "object",
        "properties": {
            "amount": {
                "type": "number",
                "description": "The amount to withdraw",
            },
            "balance": {
                "type": "number",
                "description": "The account balance before the withdraw",
            }
        },
        "required": ["amount", "balance"],
        "additionalProperties": False
    }
}

check_balance_function = {
    "name": "check_balance",
    "description": "Get the balance of the user account and display it to the user.",
    "parameters": {
        "type": "object",
        "properties": {
            "balance": {
                "type": "number",
                "description": "The account balance",
            }
        },
        "required": ["balance"],
        "additionalProperties": False
    }    
}

In [68]:
tools = [
    {"type": "function", "function": open_account_function}, 
    {"type": "function", "function": deposit_funds_function},
    {"type": "function", "function": withdraw_funds_function},
    {"type": "function", "function": check_balance_function}
]

In [70]:
def open_account(name, email):
    return {"message": f"Account opened for {name} with email {email}"}

def deposit_funds(amount, balance):
    new_balance = float(balance) + float(amount)
    return {
        "message": f"Deposited ${amount:.2f} into your account. Balance: ${new_balance:.2f}",
        "balance": new_balance
    }

def withdraw_funds(amount, balance):
    new_balance = float(balance) - float(amount)
    return {
        "message": f"Withdrew ${amount:.2f} from your account. Balance: ${new_balance:.2f}",
        "balance": new_balance
    }

def check_balance(balance):
    bal = float(balance)
    return {"message": f"Your account balance is: ${bal:.2f}", "balance": bal}

In [71]:
# We have to write that function handle_tool_call:

def handle_tool_call(assistant_msg, balance):
    tool_messages = []
    new_balance = balance

    for tc in getattr(assistant_msg, "tool_calls", []) or []:
        args = json.loads(tc.function.arguments or "{}")
        name = tc.function.name

        if name == "open_account":
            result = open_account(args.get("name"), args.get("email"))
            tool_messages.append({
                "role": "tool",
                "tool_call_id": tc.id,
                "content": json.dumps(result)  # MUST be string
            })

        elif name == "deposit_funds":
            result = deposit_funds(args["amount"], new_balance)
            new_balance = result["balance"]
            tool_messages.append({
                "role": "tool",
                "tool_call_id": tc.id,
                "content": json.dumps(result)
            })

        elif name == "withdraw_funds":
            result = withdraw_funds(args["amount"], new_balance)
            new_balance = result["balance"]
            tool_messages.append({
                "role": "tool",
                "tool_call_id": tc.id,
                "content": json.dumps(result)
            })

        elif name == "check_balance":
            result = check_balance(new_balance)
            tool_messages.append({
                "role": "tool",
                "tool_call_id": tc.id,
                "content": json.dumps(result)
            })

    return tool_messages, new_balance


In [72]:
def chat(message, history, balance):
    # Build messages from history (which already includes the welcome assistant turn)
    messages = [{"role": "system", "content": SYS_PROMPT}] + history + [
        {"role": "user", "content": message}
    ]

    resp = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    assistant_msg = resp.choices[0].message

    # If the model called tools
    if getattr(assistant_msg, "tool_calls", None):
        # 1) Append the assistant's tool-call turn as a normal message dict
        messages.append({
            "role": "assistant",
            "content": assistant_msg.content or "",
            "tool_calls": [
                {
                    "id": tc.id,
                    "type": "function",
                    "function": {
                        "name": tc.function.name,
                        "arguments": tc.function.arguments,
                    },
                } for tc in assistant_msg.tool_calls
            ],
        })

        # 2) Execute tools and extend with tool messages
        tool_msgs, balance = handle_tool_call(assistant_msg, balance)
        messages.extend(tool_msgs)  # NOT append

        # 3) Ask the model to finalize using tool results
        resp2 = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
        return resp2.choices[0].message.content, balance

    # No tools—just return the assistant text
    return assistant_msg.content, balance

# Pre-seed the Chatbot with the welcome as the first assistant message
chatbot = gr.Chatbot(
    value=[{"role": "assistant", "content": WELCOME}],
    type="messages",
    height=420
)

balance_state = gr.State(0.0)

gr.ChatInterface(
    fn=chat,
    chatbot=chatbot,   # <-- inject our pre-seeded Chatbot
    type="messages",
    additional_inputs=[balance_state],   # read current balance
    additional_outputs=[balance_state],  # write updated balance
).launch()

* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.
